In [1]:
import os
import glob
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/bimanual_wrist')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
sys.path.append('/Users/aghavamp/Desktop/Projects/AnatSearchlight')
import getpass
import importlib
from tqdm import tqdm
from pathlib import Path

import AnatSearchlight.searchlight as sl

import surfAnalysisPy as surf
import nibabel as nb
import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import seaborn as sns

import scipy.io as sio
import numpy as np
import pandas as pd

import utils
importlib.reload(utils)

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'
analysisDir = os.path.join(os.path.dirname(os.path.dirname(baseDir)), 'analysis')



## Define searchlight

In [ ]:
sn_list = [101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]
glm = 'single'
hem = ['L', 'R']
sl_name = ['left_cortex', 'right_cortex']
radius = 20


for sn in sn_list:
    try:
        # define:
        for i, h in enumerate(hem):
            mySearchlight = sl.SearchlightSurface(sl_name[i])
            surf = [os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.pial.32k.surf.gii'),
                    os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.white.32k.surf.gii')]
            mask = os.path.join(baseDir, f'glm{glm}', f's{sn}', 'mask.nii') # Mask in functional space for the subject
            roi_mask = os.path.join(atlasDir,f'ROI.32k.{h}.label.gii')

            # search light define:
            mySearchlight.define(surf,mask,roi=roi_mask,maxradius=radius)
            mySearchlight.save(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))
    
    except Exception as e:
        print(f"Error processing subject {sn}, hemisphere {h}: {e}")
        continue


## Run all searchlight

In [ ]:
sn_list = [101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]
glm = 'single'
hem = ['L', 'R']
sl_name = ['left_cortex', 'right_cortex']
radius = 20
conds = ['lhand', 'rhand', 'bi']

def mvpa_mean_function(data):
        # Example of MVPA function that returns a scalar as an output argument
        return np.mean(data)

def mvpa_function(data, **kwargs):
        Y = data
        partition_vec = kwargs['partition_vec']
        condition_vec = kwargs['condition_vec']

        G_hat,_ = pcm.est_G_crossval(Y,
                        condition_vec,
                        partition_vec,
                        X=pcm.matrix.indicator(partition_vec))
        
        D = pcm.G_to_dist(G_hat)
        n = D.shape[0]
        
        # mean of distances:
        return np.sum(D) / (n**2 - n)

for sn in sn_list:
        try:
                # define:
                for i, h in enumerate(hem):
                        mySearchlight = sl.SearchlightSurface(sl_name[i])
                        surf = [os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.pial.32k.surf.gii'),
                                os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.white.32k.surf.gii')]
                        mask = os.path.join(baseDir, f'glm{glm}', f's{sn}', 'mask.nii') # Mask in functional space for the subject
                        roi_mask = os.path.join(atlasDir,f'ROI.32k.{h}.label.gii')

                        # search light define:
                        mySearchlight.define(surf,mask,roi=roi_mask,maxradius=radius)
                        mySearchlight.save(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))

                # run mean:
                for cond in conds:
                        # regressor info:
                        reginfo = pd.read_table(os.path.join(baseDir, f'glm{glm}', f's{sn}', 'reginfo.tsv'))
                        # get indices of the regressors of interest:
                        regressors_of_interest = reginfo[reginfo.name.str.contains(cond)].index.tolist()

                        datafiles = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'beta_{i+1:04d}.nii') for i in regressors_of_interest]

                        for i, h in enumerate(hem):
                                S = sl.load(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))
                                results = S.run(datafiles, mvpa_mean_function)
                                S.data_to_cifti(results, outfilename=os.path.join(baseDir, 'searchlight', f'{cond}_s{sn}_{h}_cortex.dscalar.nii'))
        
                # run MVPA:
                for cond in conds:
                        # regressor info:
                        reginfo = pd.read_table(os.path.join(baseDir, f'glm{glm}', f's{sn}', 'reginfo.tsv'))
                        # get indices of the regressors of interest:
                        regressors_of_interest = reginfo[reginfo.name.str.contains(cond)].index.tolist()

                        datafiles = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'beta_{i+1:04d}.nii') for i in regressors_of_interest]
                        partition_vec = reginfo.loc[regressors_of_interest, 'run'].values
                        condition_vec = reginfo.loc[regressors_of_interest, 'name'].values

                        for i, h in enumerate(hem):
                                S = sl.load(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))
                                results = S.run(datafiles, mvpa_function, {'partition_vec': partition_vec, 'condition_vec': condition_vec})
                                S.data_to_cifti(results, outfilename=os.path.join(baseDir, 'searchlight', f'mvpa_{cond}_s{sn}_{h}_cortex.dscalar.nii'))
        
        except:
              print(f'Error processing subject {sn}. Skipping...')
              continue



Processing center 0 of 4473
Processing center 1000 of 4473
Processing center 2000 of 4473
Processing center 3000 of 4473
Processing center 4000 of 4473
Processing center 0 of 4473
Processing center 1000 of 4473
Processing center 2000 of 4473
Processing center 3000 of 4473
Processing center 4000 of 4473
Loading data 
Volume number 1
Volume number 2
Volume number 3
Volume number 4
Volume number 5
Volume number 6
Volume number 7
Volume number 8
Volume number 9
Volume number 10
Volume number 11
Volume number 12
Volume number 13
Volume number 14
Volume number 15
Volume number 16
Volume number 17
Volume number 18
Volume number 19
Volume number 20
Volume number 21
Volume number 22
Volume number 23
Volume number 24
Volume number 25
Volume number 26
Volume number 27
Volume number 28
Volume number 29
Volume number 30
Volume number 31
Volume number 32
Volume number 33
Volume number 34
Volume number 35
Volume number 36
Volume number 37
Volume number 38
Volume number 39
Volume number 40
Volume numb

## Marginal left hand bimanual searchlight

In [3]:
sn_list = [101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]
glm = 'single'
hem = ['L', 'R']
sl_name = ['left_cortex', 'right_cortex']
radius = 20

def mvpa_marginal_lhand_function(data, **kwargs):
        Y_all = data
        partitions_all = kwargs['partition_vec']
        conditions_all = kwargs['condition_vec']

        # define the structure of the averaged data:
        angles = [0, 60, 120, 180, 240, 300]
        partitions = np.unique(partitions_all)
        conditions = angles.copy()
        Y = np.zeros((len(partitions)*len(conditions), Y_all.shape[1]))
        partition_vec = np.repeat(partitions, len(conditions))
        condition_vec = np.tile(conditions, len(partitions))
        
        # average the activity patterns and place into Y:
        for i in range(Y.shape[0]):
            what_partition = partition_vec[i]
            what_condition = condition_vec[i]

            name = f'bi:{what_condition}'
            rows = [j for j, c in enumerate(conditions_all) if name in c and partitions_all[j] == what_partition]

            # average the activity patterns:
            Y[i, :] = np.mean(Y_all[rows, :], axis=0)
        
        # estimate the G matrix:
        G_hat,_ = pcm.est_G_crossval(Y,
                        condition_vec,
                        partition_vec,
                        X=pcm.matrix.indicator(partition_vec))
        
        D = pcm.G_to_dist(G_hat)
        n = D.shape[0]
        
        # mean of distances:
        return np.sum(D)/(n**2 - n)

for sn in sn_list:
    try:
        # run marginal MVPA:
        # regressor info:
        reginfo = pd.read_table(os.path.join(baseDir, f'glm{glm}', f's{sn}', 'reginfo.tsv'))
        # get indices of the regressors of interest:
        regressors_of_interest = reginfo[reginfo.name.str.contains('bi')].index.tolist()

        datafiles = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'beta_{i+1:04d}.nii') for i in regressors_of_interest]
        partition_vec = reginfo.loc[regressors_of_interest, 'run'].values
        condition_vec = reginfo.loc[regressors_of_interest, 'name'].values

        for i, h in enumerate(hem):
            S = sl.load(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))
            results = S.run(datafiles, mvpa_marginal_lhand_function, {'partition_vec': partition_vec, 'condition_vec': condition_vec})
            S.data_to_cifti(results, outfilename=os.path.join(baseDir, 'searchlight', f'marginal_lhand_s{sn}_{h}_cortex.dscalar.nii'))
    
    except:
            print(f'Error processing subject {sn}. Skipping...')
            continue



Loading data 
Volume number 1
Volume number 2
Volume number 3
Volume number 4
Volume number 5
Volume number 6
Volume number 7
Volume number 8
Volume number 9
Volume number 10
Volume number 11
Volume number 12
Volume number 13
Volume number 14
Volume number 15
Volume number 16
Volume number 17
Volume number 18
Volume number 19
Volume number 20
Volume number 21
Volume number 22
Volume number 23
Volume number 24
Volume number 25
Volume number 26
Volume number 27
Volume number 28
Volume number 29
Volume number 30
Volume number 31
Volume number 32
Volume number 33
Volume number 34
Volume number 35
Volume number 36
Volume number 37
Volume number 38
Volume number 39
Volume number 40
Volume number 41
Volume number 42
Volume number 43
Volume number 44
Volume number 45
Volume number 46
Volume number 47
Volume number 48
Volume number 49
Volume number 50
Volume number 51
Volume number 52
Volume number 53
Volume number 54
Volume number 55
Volume number 56
Volume number 57
Volume number 58
Volume nu

## Marginal right hand bimanual searchlight

In [2]:
sn_list = [101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]
glm = 'single'
hem = ['L', 'R']
sl_name = ['left_cortex', 'right_cortex']

def mvpa_marginal_rhand_function(data, **kwargs):
        Y_all = data
        partitions_all = kwargs['partition_vec']
        conditions_all = kwargs['condition_vec']

        # define the structure of the averaged data:
        angles = [0, 60, 120, 180, 240, 300]
        partitions = np.unique(partitions_all)
        conditions = angles.copy()
        Y = np.zeros((len(partitions)*len(conditions), Y_all.shape[1]))
        partition_vec = np.repeat(partitions, len(conditions))
        condition_vec = np.tile(conditions, len(partitions))
        
        # average the activity patterns and place into Y:
        for i in range(Y.shape[0]):
            what_partition = partition_vec[i]
            what_condition = condition_vec[i]

            name = f'_{what_condition}'
            rows = [j for j, c in enumerate(conditions_all) if c.strip().endswith(name) and partitions_all[j] == what_partition]

            # average the activity patterns:
            Y[i, :] = np.mean(Y_all[rows, :], axis=0)
        
        # estimate the G matrix:
        G_hat,_ = pcm.est_G_crossval(Y,
                        condition_vec,
                        partition_vec,
                        X=pcm.matrix.indicator(partition_vec))
        
        D = pcm.G_to_dist(G_hat)
        n = D.shape[0]
        
        # mean of distances:
        return np.sum(D)/(n**2 - n)

for sn in sn_list:
    try:
        # run marginal MVPA:
        # regressor info:
        reginfo = pd.read_table(os.path.join(baseDir, f'glm{glm}', f's{sn}', 'reginfo.tsv'))
        # get indices of the regressors of interest:
        regressors_of_interest = reginfo[reginfo.name.str.contains('bi')].index.tolist()

        datafiles = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'beta_{i+1:04d}.nii') for i in regressors_of_interest]
        partition_vec = reginfo.loc[regressors_of_interest, 'run'].values
        condition_vec = reginfo.loc[regressors_of_interest, 'name'].values

        for i, h in enumerate(hem):
            S = sl.load(os.path.join(analysisDir,'searchlight',f'sl_{h}_s{sn}.h5'))
            results = S.run(datafiles, mvpa_marginal_rhand_function, {'partition_vec': partition_vec, 'condition_vec': condition_vec})
            S.data_to_cifti(results, outfilename=os.path.join(baseDir, 'searchlight', f'marginal_rhand_s{sn}_{h}_cortex.dscalar.nii'))
    
    except:
        print(f'Error processing subject {sn}. Skipping...')
        continue



Loading data 
Volume number 1
Volume number 2
Volume number 3
Volume number 4
Volume number 5
Volume number 6
Volume number 7
Volume number 8
Volume number 9
Volume number 10
Volume number 11
Volume number 12
Volume number 13
Volume number 14
Volume number 15
Volume number 16
Volume number 17
Volume number 18
Volume number 19
Volume number 20
Volume number 21
Volume number 22
Volume number 23
Volume number 24
Volume number 25
Volume number 26
Volume number 27
Volume number 28
Volume number 29
Volume number 30
Volume number 31
Volume number 32
Volume number 33
Volume number 34
Volume number 35
Volume number 36
Volume number 37
Volume number 38
Volume number 39
Volume number 40
Volume number 41
Volume number 42
Volume number 43
Volume number 44
Volume number 45
Volume number 46
Volume number 47
Volume number 48
Volume number 49
Volume number 50
Volume number 51
Volume number 52
Volume number 53
Volume number 54
Volume number 55
Volume number 56
Volume number 57
Volume number 58
Volume nu

## Open mean in wb_view

In [46]:
import subprocess

sn = 101

conds = ['rhand', 'lhand', 'bi']
hem = ['L', 'R']

# add paths:
paths = []
# add surface files:
for h in hem:
    paths.append(os.path.abspath(os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.inflated.32k.surf.gii')))

# add searchlight results:
for h in hem:
    for cond in conds:
        paths.append(os.path.abspath(os.path.join(baseDir, 'searchlight', f'{cond}_s{sn}_{h}_cortex.dscalar.nii')))



# Run wb_view:
subprocess.run(['wb_view', paths[0], paths[1], paths[2], paths[3], paths[4], paths[5], paths[6], paths[7]])





Info: Resources loaded:
   :/About   :/BalsaUploadDialog   :/Cursor   :/DingOntology   :/Fonts   :/general_resources.qrc   :/help_resources.qrc   :/HelpFiles   :/LayersPanel   :/MessageDialog   :/PaletteEditorDialog   :/PaletteSettings   :/qpdf   :/qt-project.org   :/RecentFilesDialog   :/SceneFileDialog   :/SpecFileDialog   :/Splash   :/ToolBar   :/update_resources.sh


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.L.inflated.32k.surf.gii was 0.028059 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.R.inflated.32k.surf.gii was 0.0269 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/rhand_s101_L_cortex.dscalar.nii was 0.006142 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/lhand_s101_L_cortex.dscalar.nii was 0.002765 seconds.


Info: Time to read /Users/aghavamp/Desktop/Proj

CompletedProcess(args=['wb_view', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.L.inflated.32k.surf.gii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/surfaceWB/s101/s101.R.inflated.32k.surf.gii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/rhand_s101_L_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/lhand_s101_L_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/bi_s101_L_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/rhand_s101_R_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/lhand_s101_R_cortex.dscalar.nii', '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/searchlight/bi_s101_R_cortex.dscalar.nii'], returncode=0)